##### 1. 导入需要的库和数据

In [9]:
import gensim
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
# from gensim.parsing.preprocessing import preprocess_string,remove_stopwords
import jieba
import pandas as pd
import numpy as np
import random
# import warnings
# warnings.filterwarnings("ignore")

In [10]:
#读取两张csv表，并进行合并处理
df1 = pd.DataFrame(pd.read_csv('./first sheet v2.csv'))
df2 = pd.DataFrame(pd.read_csv('./second sheet v2.csv'))
df_total = pd.concat([df1,df2],ignore_index=True)
df_practice = df_total[['source', 'content']]

##### 2. 获取用来训练的评论数据，逐条进行分停词，然后训练doc2vec模型

##### 2.1 获取评论数据

In [11]:
#仅保留评论数据
def getText():
    discuss_train = list(df_practice['content'])
    return discuss_train
Text = getText()
# Text

In [34]:
#前两条
print(Text[0:2])

['人工智能后的社会有两个关键词一个是高效，另一个是透明。高效就不用说了，透明才是最有意思的事情。人永远不可能完全透明，人有隐私有不可明说的事情。就必须有东西挡着点，所以就必须有衣服来遮羞，但凡有遮遮掩掩的地方，就容易藏污纳垢，有暗箱操作，即使你没有暗箱操作人们也会认为你有，不然为什么要用衣服来遮挡。 所以只要智商够的你们就懂了吧，好好想想。光这些就剩多少钱。还有百年后重新回头发现手机早晚会成为最反人类的科技产品', '人工智能在当代社会已经是一个不可阻拦的发展大趋势，而且人工智能的影响和运用也深入到了社会生活等方方面面，对人类的衣食住行产生了巨大的改变，同时也在改变着传统或者现代的产业结构和人员配置。\n\n人类生活的各个行业例如农业、体育、医疗卫生、制造业、律师行业、记者和编辑行业等领域都已经在或者将会在未来深入使用人工智能技术，这对于未来世界的改变是巨大而且无法想象的。\n\n在未来几年内，机器人与人工智能能给世界带来的影响将远远超过个人计算和互联网在过去三十年间已经对世界所造成的改变。人工智能将成为未来年内的产业新风口，像I安钱电力彻底颠覆人类世界一样，人工智能也必将掀起一场新的而且持续深入的产业革命。\n\n但是事情的发展总是两面性的，人工智能的发展和百年前的工业革命一样将会在很大程度上造成劳动力的转换，在这个过程中，将会出现一系列的问题，而这些问题很有可能成为阻碍人工智能继续发展的巨大阻力。人工智能领域的最新发展对科技变化的促进作用可能会以两种基本的方式搅乱我们的劳动市场。首先，大部分自动化作业都会代替工人，从而减少工作的机会，这就意味着血药人工作的地方变得更少，这种威胁显而易见，也很容易度量；另外，很多科技进步会通过让商家重组和重建运营的方式来改变游戏规则，这样的组织精华和流程不仅经常会淘汰工作岗位，也会淘汰技能。\n\n但从总体上来说，人工智能所带给未来人类世界的好处是要大于其弊端的，而且在未来人类生活的理想蓝图中，人工智能也会发挥着很大的作用和推动力，这是一个必然也无法阻止的趋势。\n获得更多创业融资干货，也可直接点击：\n创业融资没经验，提交留言获指导！\u200bySRdRsM专业老师对免费咨询提交留言。【免费获得】：份商业计划书模板、个投资人名单、份企业经营日常合同、份行业分析报告。年行业经验老师对指导，助你成功敲开投资人的大门。']


##### 2.2 分、停词，进行基本的清洗

In [12]:
#对评论进行分词、停词操作,为了后期抽样和可视化的便利，本函数每次仅处理一条评论
#这里的停用词表应该和项目之前的有差异，后期需更换停用词表！！！
def cut_sentence(text):
    stop_list = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
    stop_list.append("\n")
    each_cut = jieba.lcut(text)
    each_result = [word for word in each_cut if word not in stop_list]
    each_result_str =" ".join(str(i) for i in each_result) 
    return each_result_str

In [13]:
#给定一个用来训练句向量的序号列表，可以用来选择使用的训练语料
#前500条
train_num = [item for item in range(67248)]

In [14]:
#对用来训练的评论，逐句进行分、停词
Text_Seg = list()#将清洗完的评论，以字符串形式，放入列表储存
for item in train_num:
    Text_Seg.append(cut_sentence(Text[item]))


In [33]:
# 展示清洗后的数据
print(Text_Seg[0:2])

['人工智能 社会 两个 关键词 高效 透明 高效 不用说 透明 有意思 事情 永远 透明 人有 隐私 明说 事情 东西 挡 点 衣服 遮羞 遮遮掩掩 地方 藏污纳垢 暗箱 操作 暗箱 操作 衣服 遮挡   智商 够 懂 好好 想想 剩 钱 百年 回头 发现 手机 早晚 最反 人类 科技 产品', '人工智能 当代 社会 阻拦 发展 趋势 人工智能 影响 社会 生活 方方面面 人类 衣食住行 改变 改变 传统 产业结构 人员 配置 人类 生活 行业 农业 体育 医疗卫生 制造业 律师 行业 记者 编辑 行业 领域 将会 未来 人工智能 技术 未来 世界 改变 想象 未来 几年 机器人 人工智能 世界 带来 影响 远远 超过 计算 互联网 三十年 间 世界 改变 人工智能 未来 年内 产业 新风 口 I 安钱 电力 颠覆 人类 世界 人工智能 掀起 一场 新 持续 产业革命 事情 发展 两面性 人工智能 发展 百年 前 工业革命 将会 很大 程度 劳动力 转换 过程 中 将会 一系列 阻碍 人工智能 发展 阻力 人工智能 领域 最新 发展 科技 变化 促进作用 会以 两种 方式 搅乱 劳动 市场 大部分 自动化 作业 工人 减少 工作 机会 意味着 血药 工作 地方 变得 少 威胁 显而易见 度量 科技进步 商家 重组 重建 运营 方式 改变 游戏规则 组织 精华 流程 淘汰 工作岗位 淘汰 技能 从总体上 人工智能 带给 未来 人类 世界 好处 大于 弊端 未来 人类 生活 理想 蓝图 中 人工智能 发挥 很大 作用 推动力 这是 阻止 趋势 创业 融资 干货 点击 创业 融资 经验 提交 留言 获 指导 \u200b ySRdRsM 专业 老师 免费 咨询 提交 留言 免费 份 商业 计划书 模板 投资人 名单 份 企业 经营 日常 合同 份 行业 分析 报告 年 行业 经验 老师 指导 助 成功 敲开 投资人 大门']


##### 2.3 将评论转化为符合训练要求的格式

In [15]:
#将句子转化为符合gensim.models.Doc2Vec库要求的格式
TaggededDocument = gensim.models.doc2vec.TaggedDocument
def X_train(seg_Text):
    x_train = []
    for i,text in enumerate(seg_Text):
        word_list = text.split(' ')
        length = len(word_list)
        word_list[length-1]=word_list[length-1].strip()
        document = TaggededDocument(word_list,tags=[i])
        x_train.append(document)
    return x_train

#Text_pred保存了格式规范化后的训练数据
Text_pred = X_train(Text_Seg)

In [36]:
#展示处理后的格式
print(Text_pred[0:2])

[TaggedDocument(words=['人工智能', '社会', '两个', '关键词', '高效', '透明', '高效', '不用说', '透明', '有意思', '事情', '永远', '透明', '人有', '隐私', '明说', '事情', '东西', '挡', '点', '衣服', '遮羞', '遮遮掩掩', '地方', '藏污纳垢', '暗箱', '操作', '暗箱', '操作', '衣服', '遮挡', '', '', '智商', '够', '懂', '好好', '想想', '剩', '钱', '百年', '回头', '发现', '手机', '早晚', '最反', '人类', '科技', '产品'], tags=[0]), TaggedDocument(words=['人工智能', '当代', '社会', '阻拦', '发展', '趋势', '人工智能', '影响', '社会', '生活', '方方面面', '人类', '衣食住行', '改变', '改变', '传统', '产业结构', '人员', '配置', '人类', '生活', '行业', '农业', '体育', '医疗卫生', '制造业', '律师', '行业', '记者', '编辑', '行业', '领域', '将会', '未来', '人工智能', '技术', '未来', '世界', '改变', '想象', '未来', '几年', '机器人', '人工智能', '世界', '带来', '影响', '远远', '超过', '计算', '互联网', '三十年', '间', '世界', '改变', '人工智能', '未来', '年内', '产业', '新风', '口', 'I', '安钱', '电力', '颠覆', '人类', '世界', '人工智能', '掀起', '一场', '新', '持续', '产业革命', '事情', '发展', '两面性', '人工智能', '发展', '百年', '前', '工业革命', '将会', '很大', '程度', '劳动力', '转换', '过程', '中', '将会', '一系列', '阻碍', '人工智能', '发展', '阻力', '人工智能', '领域', '最新', '发展', '科技', '变化', '促进作用', '会以', '两种',

##### 2.4 训练模型

In [37]:
#doc2vec正式训练
model = gensim.models.Doc2Vec(vector_size=300, window=10, min_count=5,
                            workers=4, alpha=0.025, min_alpha=0.025, epochs=12)
model.build_vocab(Text_pred)
print("开始训练...")
# 训练模型
model.train(Text_pred, total_examples=model.corpus_count, epochs=12)
model.save("doc2vec.model")
print("模型已保存")

KeyboardInterrupt: 

##### 2.5输出对应向量

In [17]:
#根据训练得到的模型，将文本转化为向量
def sent2vec(model, words):
    vect_list = []
    for w in words:
        try:
            vect_list.append(model.wv[w])
        except:
            continue
    vect_list = np.array(vect_list)
    vect = vect_list.sum(axis=0)
    return vect / np.sqrt((vect ** 2).sum()) 

In [18]:
#将句向量结果用Dataframe格式保存
doc_num2vec_dict = dict() #创建一个序号对应向量的字典
for item in range(67248): 
    doc_num2vec_dict[item] = sent2vec(model,Text[item])
    
matrix = pd.DataFrame() #用Dataframe格式保存
for i in range(67248):
    try:
        array_i = pd.DataFrame(sent2vec(model,Text[i])).T
        matrix = pd.concat([matrix,array_i],axis=0,ignore_index=True)
    except:
        array_i = pd.DataFrame(np.zeros(300)).T
        matrix = pd.concat([matrix,array_i],axis=0,ignore_index=True)

In [27]:
matrix.to_csv("matrix.csv")

##### 3.降维与可视化分析

##### 3.1PCA降维

In [20]:
#PCA降维
from sklearn.decomposition import PCA
pca = PCA(n_components=2) 
pca = pca.fit(matrix)  # 拟合模型
matrix_ld = pd.DataFrame(pca.transform(matrix))  # 获取降维后的新矩阵 #low dimensions

##### 3.2对matrix_ld矩阵补充平台来源属性

In [23]:
#获取评论对应的平台编码
df_total = pd.concat([df1,df2],ignore_index=True)
label = pd.DataFrame(df_total)
subtotal = label.groupby(['source']).count()
print(subtotal)
#平台来源编码
label.loc[label['source']=="微博",['source']] = 0 
label.loc[label['source']=="知乎问答",['source']] = 1
label.loc[label['source']=="简书文章",['source']] = 2 
label.loc[label['source']=="豆瓣日记",['source']] = 3 

def getSource():
    label_targets = label['source'] #label_targets用来表示平台来源
    label_targets = pd.DataFrame(label_targets)
    return pd.DataFrame(label_targets)
# 加入source的编码
matrix_visualization = pd.concat([matrix_ld,getSource()],axis=1,ignore_index=False)

        Unnamed: 0  link  title_link  title_link-href  topics  author  \
source                                                                  
微博               0     0           0                0       0   51442   
知乎问答             0  7019        7019             7019    7019    7019   
简书文章             0  6474        6474             6474       0    6474   
豆瓣日记             0  2313        2313             2313       0    2313   

        author_info  content  support   time  
source                                        
微博                0    51442        0  51442  
知乎问答           4719     7019        0   7019  
简书文章              0     6474     6474   6474  
豆瓣日记              0     2313      505   2313  


In [28]:
matrix_visualization.to_csv("matrix_visualization.csv")

##### 3.3可视化

In [ ]:
#可视化
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  #导入绘图操作用到的库
import random

#读取数据得到一个字典类型数据，需要根据键名 ’data‘ 取出对应的值。
data  = matrix_visualization
fig = plt.figure(dpi=1000)  # 创建画布
ax = fig.add_subplot(111)


idx_0 = data[data['source'] == 0].index.to_list()  # 找出标签为0的样本行数
ns_0 = round(len(idx_0)/73)
samples_0=random.sample(idx_0,ns_0)#随机抽取1/10

idx_1 = data[data['source'] == 1].index.to_list()  # 找出标签为1的样本行数
ns_1 = round(len(idx_1)/10)
samples_1=random.sample(idx_1,ns_1)#随机抽取1/10

idx_2 = data[data['source'] == 2].index.to_list()  # 找出标签为2的样本行数
ns_2 = round(len(idx_2)/10)
samples_2=random.sample(idx_2,ns_2)#随机抽取1/10

idx_3 = data[data['source'] == 3].index.to_list()  # 找出标签为3的样本行数
ns_3 = round(len(idx_3)/3)
samples_3=random.sample(idx_3,ns_3)#随机抽取1/10

#绘制散点图
p0 = ax.scatter(data.loc[samples_0,'PCA1'], data.loc[samples_0,'PCA2'], marker='.', color='r', s=0.5,label = 'Weibo')
p1 = ax.scatter(data.loc[samples_1,'PCA1'], data.loc[samples_1,'PCA2'], marker='.', color='g', s=0.5,label = 'Zhihu')
p2 = ax.scatter(data.loc[samples_2,'PCA1'], data.loc[samples_2,'PCA2'], marker='.', color='y', s=0.5,label = 'Jianshu')
p3 = ax.scatter(data.loc[samples_3,'PCA1'], data.loc[samples_3,'PCA2'], marker='.', color='b', s=0.5,label = 'Douban')

ax.legend() 
plt.show()   #显示散点图